# MNIST con PyTorch: CPU vs GPU

## MNIST

MNIST es una dataset de dígitos escritos a mano (disponible en http://yann.lecun.com/exdb/mnist/). Tiene un conjunto de entrenamiento de 60.000 ejemplos y un conjunto de prueba de 10.000 ejemplos. Es un subconjunto de un conjunto más grande disponible en NIST. Los dígitos se normalizaron en tamaño y se centraron en una imagen de tamaño fijo.

Según sus autores, es una buena base de datos para las personas que desean probar técnicas de aprendizaje y métodos de reconocimiento de patrones en datos del mundo real mientras dedican un esfuerzo mínimo al preprocesamiento y formato.

## PyTorch

El uso de Deep Learning ha crecido enormemente en los últimos años con el aumento de GPU, big data, proveedores de nube como Amazon Web Services (AWS) y Google Cloud, y frameworks como Torch, TensorFlow, Caffe y PyTorch. Además, las grandes empresas comparten algoritmos entrenados en enormes conjuntos de datos, lo que ayuda a las nuevas empresas a construir sistemas de vanguardia en varios casos de uso con poco esfuerzo.

Con PyTorch, al igual que con otros frameworks, una vez que tengamos los datos en forma de tensores, podemos hacer operaciones como suma, resta, multiplicación, producto escalar y multiplicación de matrices. Todas estas operaciones es posible realizarlas con el uso de la CPU o en la GPU. PyTorch proporciona una función simple llamada `cuda` para copiar un tensor de la CPU a la GPU.

PyTorch utiliza ampliamente conceptos de Python, como clases, estructuras y bucles condicionales, lo que nos permite construir algoritmos DL de una manera puramente orientada a objetos. La mayoría de los otros marcos populares traen su propio estilo de programación, lo que a veces dificulta la escritura de nuevos algoritmos y no admite la depuración intuitiva.

Echaremos un vistazo a algunas de las operaciones y compararemos el rendimiento entre las operaciones de multiplicación de matrices en la CPU y la GPU, para luego, comparar el rendimiento al entrenar una red neuronal utilizando el dataset MNIST.

# Intalación del driver NVidia, Cuda y PyTorch correspondiente

Una vez realizada una introducción es hora de irnos a una parte más técnica y específica. Trataré de mencionar o describir algunos pasos que pudieran convertirse en una ayuda para otras personas que comienzan en este mundo. Me permitiré hacerlo de una forma menos formal y más directa, al fin al cabo esto no es más que un resumen de las horas y GBs (paquetes de 2.5 GB LTE que brida ETECSA) invertidos en aras de tener un ecosistema listo para trabajar.

Partiré mencionando el hardware y software con los que cuento para luego resumir en una serie de pasos la instalación del driver NVidia, Cuda y PyTorch.

Hardware:
- CPU: Intel(R) Celeron(R) CPU G1830 @ 2.80GHz
- RAM: 4GB DIMM DDR3 Synchronous 1333 MHz (0,8 ns)
- GPU: GeForce GTX 780 Ti

Sistema Operativo:
- GNU/Linux (Linux Mint 19.1 https://linuxmint.com/) 64 bits

Ya llega la hora de ir para arriba del lío y SI, a enfrentarnos al bloqueo. Estamos en presencia de una de los tantas dificultades que tenemos a causa del bloqueo: descargar el driver correspondiente a nuestra tarjeta de video. Lo sencillo y normal sería ir a https://www.nvidia.com/Download/index.aspx pero ahí solo encontraremos un `Access Denied. You don't have permission to access "http://www.nvidia.com/Download/index.aspx" on this server.` Por lo tanto me redirijo a una VPN y descago el archivo `NVIDIA-Linux-x86_64-410.93.run`. Lo otro es sencillo:

- `$ sudo su`
- `$ sh NVIDIA-Linux-x86_64-410.93.run`

![](images/driver.png)

Una vez ya instalado el driver sigo obligado a utilizar VPN para hacerme del archivo `cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64.deb`. CUDA es una plataforma de computación paralela y un modelo de programación inventado por NVIDIA. Permite aumentos drásticos en el rendimiento informático al aprovechar la potencia de la unidad de procesamiento de gráficos (GPU).

Al fin llegamos a PyTorch, por suerte aún nos permiten las descargas desde https://download.pytorch.org/whl/torch_stable.html, por lo que nos descargamos `torch-1.3.0+cu100-cp36-cp36m-linux_x86_64.whl` ya que tengo Python 3.6.7. No recomiendo instalar una versión más actualizada porque no funcionarán algunos codes a causa de la incompatibilidad con la GPU (GeForce GTX 780 Ti).

Ahora sí. Listos para escribir código. Probemos si realmente estamos utilizando la GPU.

In [5]:
import torch

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


En efecto, si al ejecuar le aparece `cuda:0` estamos utilizando la GPU, de lo contrario, si le aparece `cpu` es porque solo se estaría utilizando la CPU.

Ahora comparemos ambos casos al multiplicar 2 matrices de tensores y veamos los tiempos. Tenga en cuenta que cualquier tensor se puede mover a la GPU llamando a la función .cuda ().

Creamos dos matrices 10000 x 10000:

In [14]:
a = torch.rand(10000, 10000)
b = torch.rand(10000, 10000)

In [15]:
from time import time

Multipliquemos `a` x `b` en la CPU:

In [16]:
start_time = time()
a.matmul(b)
elapsed_time = time() - start_time
print("Elapsed time: %0.10f seconds." % elapsed_time)

Elapsed time: 52.0623407364 seconds.


Mueva los tensores a la GPU

In [18]:
if torch.cuda.is_available():
    start_time = time()
    a = a.cuda()
    b = b.cuda()
    a.matmul(b)
    elapsed_time = time() - start_time
else:
    print("Cuda not available")
print("Elapsed time: %0.10f seconds." % elapsed_time)

Elapsed time: 0.0085325241 seconds.


En esta corrida la multiplicación de las matrices generadas en la CPU fue de 52 segundos, mientras que la GPU lo calculó en 0.4 segundos. Ummmmmm que interesante!!!  

Veamos que sucede al entrenar una red utilizando el dataset MNIST realizando solo 5 iteraciones (epochs).

CPU:

In [23]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/mnist_data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/mnist_data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

start_time = time()

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

elapsed_time = time() - start_time
print("Elapsed time (with CPU): %0.10f seconds." % elapsed_time)

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1138
Epoch [1/5], Step [200/600], Loss: 0.1962
Epoch [1/5], Step [300/600], Loss: 0.0515
Epoch [1/5], Step [400/600], Loss: 0.0622
Epoch [1/5], Step [500/600], Loss: 0.0561
Epoch [1/5], Step [600/600], Loss: 0.0321
Epoch [2/5], Step [100/600], Loss: 0.0760
Epoch [2/5], Step [200/600], Loss: 0.1218
Epoch [2/5], Step [300/600], Loss: 0.0151
Epoch [2/5], Step [400/600], Loss: 0.0569
Epoch [2/5], Step [500/600], Loss: 0.0924
Epoch [2/5], Step [600/600], Loss: 0.0084
Epoch [3/5], Step [100/600], Loss: 0.0275
Epoch [3/5], Step [200/600], Loss: 0.0058
Epoch [3/5], Step [300/600], Loss: 0.1034
Epoch [3/5], Step [400/600], Loss: 0.1626
Epoch [3/5], Step [500/600], Loss: 0.0463
Epoch [3/5], Step [600/600], Loss: 0.0949
Epoch [4/5], Step [100/600], Loss: 0.0052
Epoch [4/5], Step [200/600], Loss: 0.0093
Epoch [4/5], Step [300/600], Loss: 0.0427
Epoch [4/5], Step [400/600], Loss: 0.0344
Epoch [4/5], Step [500/600], Loss: 0.0222
Epoch [4/5], Step [600/600], Loss:

GPU:

In [24]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cuda:0'

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/mnist_data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/mnist_data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

start_time = time()

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

elapsed_time = time() - start_time
print("Elapsed time (with GPU): %0.10f seconds." % elapsed_time)

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.0821
Epoch [1/5], Step [200/600], Loss: 0.0578
Epoch [1/5], Step [300/600], Loss: 0.0206
Epoch [1/5], Step [400/600], Loss: 0.0725
Epoch [1/5], Step [500/600], Loss: 0.0339
Epoch [1/5], Step [600/600], Loss: 0.0549
Epoch [2/5], Step [100/600], Loss: 0.0266
Epoch [2/5], Step [200/600], Loss: 0.0145
Epoch [2/5], Step [300/600], Loss: 0.0972
Epoch [2/5], Step [400/600], Loss: 0.0338
Epoch [2/5], Step [500/600], Loss: 0.0581
Epoch [2/5], Step [600/600], Loss: 0.0249
Epoch [3/5], Step [100/600], Loss: 0.0145
Epoch [3/5], Step [200/600], Loss: 0.0591
Epoch [3/5], Step [300/600], Loss: 0.0166
Epoch [3/5], Step [400/600], Loss: 0.0322
Epoch [3/5], Step [500/600], Loss: 0.1136
Epoch [3/5], Step [600/600], Loss: 0.0233
Epoch [4/5], Step [100/600], Loss: 0.0272
Epoch [4/5], Step [200/600], Loss: 0.0026
Epoch [4/5], Step [300/600], Loss: 0.0017
Epoch [4/5], Step [400/600], Loss: 0.0166
Epoch [4/5], Step [500/600], Loss: 0.0114
Epoch [4/5], Step [600/600], Loss:

Pues sencillo, utilizando CPU algo más de 13 minutos y con la GPU menos de 1 minuto.